# 🤝 Enterprise Multi-Agent Workflow Systems (.NET)

## 📋 Learning Objectives

This notebook demonstrates how to build sophisticated enterprise-grade multi-agent systems using the Microsoft Agent Framework in .NET with GitHub Models. You'll learn to orchestrate multiple specialized agents working together through structured workflows, leveraging .NET's enterprise features for production-ready solutions.

**Enterprise Multi-Agent Capabilities You'll Build:**
- 👥 **Agent Collaboration**: Type-safe agent coordination with compile-time validation
- 🔄 **Workflow Orchestration**: Declarative workflow definition with .NET's async patterns
- 🎭 **Role Specialization**: Strongly-typed agent personalities and expertise domains
- 🏢 **Enterprise Integration**: Production-ready patterns with monitoring and error handling


## ⚙️ Prerequisites & Setup

**Development Environment:**
- .NET 9.0 SDK or higher
- Visual Studio 2022 or VS Code with C# extension
- Azure subscription (for persistent agents)

**Required NuGet Packages:**
```xml
<PackageReference Include="Microsoft.Extensions.AI.Abstractions" Version="9.9.0" />
<PackageReference Include="Azure.AI.Agents.Persistent" Version="1.2.0-beta.4" />
<PackageReference Include="Azure.Identity" Version="1.15.0" />
<PackageReference Include="System.Linq.Async" Version="6.0.3" />
<PackageReference Include="Microsoft.Extensions.AI" Version="9.8.0" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
```



In [1]:
// 📦 NuGet Package Reference: Microsoft Extensions AI Abstractions
// Core AI abstraction layer providing standardized interfaces for multi-agent systems
// Essential for building type-safe, enterprise-grade agent orchestration
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.9.0"

Installed Packages Microsoft.Extensions.AI.Abstractions, 9.9.0

In [2]:
// 📦 Enterprise Multi-Agent Dependencies
// Azure.AI.Agents.Persistent: Enterprise agent management with state persistence and Azure integration
// Azure.Identity: Secure authentication for Azure services in multi-agent workflows
// System.Linq.Async: High-performance asynchronous LINQ operations for agent data processing
// Microsoft.Extensions.AI: Core AI service abstractions for agent coordination
// DotNetEnv: Secure environment variable management for multi-agent configuration
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.4"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: Microsoft.Extensions.AI, 9.8.0"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: OpenTelemetry.Api, 1.0.0"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.4 Azure.Identity, 1.15.0 DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.8.0 OpenTelemetry.Api, 1.0.1 System.Linq.Async, 6.0.3

In [3]:
// 🔗 Local Assembly Reference: Microsoft Agents AI OpenAI
// OpenAI-compatible integration for GitHub Models in multi-agent systems
// Enables high-performance language model access for agent communication and reasoning
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0\Microsoft.Agents.AI.OpenAI.dll"


In [4]:
// 🔄 Local Assembly Reference: Microsoft Agents Workflows
// Advanced workflow orchestration engine for complex multi-agent coordination
// Provides declarative workflow definition and type-safe agent interaction patterns
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.Workflows\bin\Debug\net9.0\Microsoft.Agents.AI.Workflows.dll"

In [5]:
// 🔌 NuGet Package Reference: OpenAI Integration (Preview)
// Preview version providing GitHub Models compatibility for multi-agent workflows
// Enables seamless integration with GitHub's model inference service for agent communication
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [6]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [7]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;
using OpenAI;

In [8]:
 using DotNetEnv;

In [9]:
Env.Load("../../../.env");

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
const string REVIEWER_NAME = "Concierge";
const string REVIEWER_INSTRUCTIONS = @"""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """;

In [14]:
const string FRONTDESK_NAME = "FrontDesk";
const string FRONTDESK_INSTRUCTIONS = @"""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """;

In [15]:

ChatClientAgentOptions frontdeskAgentOptions = new(name: FRONTDESK_NAME, instructions: FRONTDESK_INSTRUCTIONS);
ChatClientAgentOptions reviewerAgentOptions = new(name: REVIEWER_NAME, instructions: REVIEWER_INSTRUCTIONS);

In [16]:
AIAgent reviewerAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    reviewerAgentOptions);
AIAgent frontdeskAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    frontdeskAgentOptions);

In [17]:
var workflow = new WorkflowBuilder(frontdeskAgent)
            .AddEdge(frontdeskAgent, reviewerAgent)
            .Build();

In [18]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, new ChatMessage(ChatRole.User, "I would like to go to Paris."));

In [19]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));

In [20]:
string strResult = "";

In [21]:
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
            if (evt is AgentRunUpdateEvent executorComplete)
            {
                strResult += executorComplete.Data;
                Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
            }
}

6c315925a6644422a6223c37e92bbcab: 
6c315925a6644422a6223c37e92bbcab: Visit
6c315925a6644422a6223c37e92bbcab:  the
6c315925a6644422a6223c37e92bbcab:  Louvre
6c315925a6644422a6223c37e92bbcab:  Museum
6c315925a6644422a6223c37e92bbcab:  for
6c315925a6644422a6223c37e92bbcab:  its
6c315925a6644422a6223c37e92bbcab:  impressive
6c315925a6644422a6223c37e92bbcab:  art
6c315925a6644422a6223c37e92bbcab:  collection
6c315925a6644422a6223c37e92bbcab: .
6c315925a6644422a6223c37e92bbcab: 
6c315925a6644422a6223c37e92bbcab: 
0cc3178682e34ee6af182fa6983f57e3: 
0cc3178682e34ee6af182fa6983f57e3: This
0cc3178682e34ee6af182fa6983f57e3:  recommendation
0cc3178682e34ee6af182fa6983f57e3:  may
0cc3178682e34ee6af182fa6983f57e3:  not
0cc3178682e34ee6af182fa6983f57e3:  be
0cc3178682e34ee6af182fa6983f57e3:  the
0cc3178682e34ee6af182fa6983f57e3:  best
0cc3178682e34ee6af182fa6983f57e3:  non
0cc3178682e34ee6af182fa6983f57e3: -tour
0cc3178682e34ee6af182fa6983f57e3: isty
0cc3178682e34ee6af182fa6983f57e3:  experience
0cc3

In [22]:
strResult

Visit the Louvre Museum for its impressive art collection.This recommendation may not be the best non-touristy experience, as the Louvre is one of the most visited attractions in the world. To refine the recommendation, focus on experiences that highlight local culture, art, and history in a more intimate setting, perhaps by exploring lesser-known galleries or community art projects that allow travelers to engage with local artists and traditions.